In [1]:
# armazenar os documentos totais em variavel

import os
# rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'

docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [2]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):
    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)
    

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 1]
    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)

    
    return text



In [4]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [5]:
preprocessed_documents = preprocess_collection(docs)

In [6]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
rootdir = '/media/developer/DATA/doutorado/rec informacao/mini_newsgroups'

docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [7]:
doc_consultas_preproc = preprocess_collection(docs_consultas)

In [10]:
doc_consultas_preproc[150]

{'tema': 'comp.graphics',
 'nome_arquivo': '39008',
 'conteudo': 'xref cantaloupe.srv.cs.cmu.edu alt.binaries.pictures.supermodels:949 alt.graphics.pixutils:5400 comp.graphics:39008 alt.binaries.pictures.erotica.d:15194 alt.binaries.pictures.d:8920 path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu cis.ohio-state.edu zaphod.mps.ohio-state.edu usc elroy.jpl.nasa.gov cougar ressler ressler cougar.jpl.nasa.gov mike ressler newsgroups alt.binaries.pictures.supermodels alt.graphics.pixutils comp.graphics alt.binaries.pictures.erotica.d alt.binaries.pictures.d subject stop argument xv-3.00 24-bit oh make date 30 apr 1993 20:31:21 gmt organization jet propulsion lab mid-infrared camera project line 13 message-id reference nntp-posting-host cougar.jpl.nasa.gov stop stop stop stop argument getting silly xv-3.00 handle 24-bit image without quantizing argument moot null void please stop quibbling obsolete version program besides generate ton verbiage new shareware licensing

In [8]:
import math
import numpy as np

In [10]:
# Função para calcular o TF (Term Frequency) de um termo em um documento
def calcular_tf(termo, documento):
    palavras_documento = documento.lower().split()
    frequencia = palavras_documento.count(termo.lower())
    return frequencia / len(palavras_documento)

In [9]:
# Função para calcular a similaridade de cosseno entre dois vetores
def calcular_similaridade_cosseno(vetor1, vetor2):
    numerador = np.dot(vetor1, vetor2)
    denominador = np.linalg.norm(vetor1) * np.linalg.norm(vetor2)
    return numerador / denominador

In [12]:
temas = [doc["tema"] for doc in doc_consultas_preproc]
conteudos = [doc["conteudo"] for doc in doc_consultas_preproc]

# Construir o vocabulário
vocabulario = set()
for documento in conteudos:
    palavras = documento.lower().split()
    vocabulario.update(palavras)

vocabulario = list(vocabulario)
vocabulario.sort()
num_termos = len(vocabulario)

# Pré-calcular o IDF para cada termo
idf_dict = {}
for termo in vocabulario:
    num_documentos_com_termo = sum(1 for documento in conteudos if termo.lower() in documento.lower())
    idf_dict[termo] = math.log(len(conteudos) / (1 + num_documentos_com_termo))

# Calcular a matriz TF-IDF
tfidf_matrix = np.zeros((len(conteudos), num_termos))
for i, documento in enumerate(conteudos):
    palavras_documento = documento.lower().split()
    for j, termo in enumerate(vocabulario):
        tf = palavras_documento.count(termo.lower()) / len(palavras_documento)
        idf = idf_dict[termo]
        tfidf_matrix[i, j] = tf * idf



In [ ]:
input_documento = doc_consultas_preproc[150]['conteudo']

# Calcular o vetor TF-IDF para o documento de entrada
input_vetor_tfidf = np.zeros(num_termos)
palavras_input = input_documento.lower().split()
for j, termo in enumerate(vocabulario):
    tf = palavras_input.count(termo.lower()) / len(palavras_input)
    idf = idf_dict[termo]
    input_vetor_tfidf[j] = tf * idf

# Calcular a similaridade de cosseno entre o documento de entrada e cada documento existente
similaridades = np.zeros(len(conteudos))
for i, vetor_tfidf in enumerate(tfidf_matrix):
    similaridades[i] = calcular_similaridade_cosseno(input_vetor_tfidf, vetor_tfidf)

resultados = list(zip(similaridades, temas, conteudos))
resultados.sort(reverse=True)

for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
    print(f"Rank {rank}: Similaridade {similaridade:.4f} - Tema: {tema}")
    print(f"Conteúdo: {conteudo}")
    print()

In [28]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

result_final = {}
for d in dirs_abs:
    result_final[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(result_final)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [20]:
def calcula_map(resultados, tema_verificado):
    acertos = 0
    maps = []
    #for i, doc in enumerate(ranking):
    for i, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if tema_verificado == tema:
            acertos +=1
            maps.append(acertos/(i+1))
    return mean(maps)

In [33]:
from statistics import mean
from IPython.display import clear_output

p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, input_documento in enumerate(doc_consultas_preproc):
    
    if c == 0:
        tema_atual = input_documento['tema']
    
    acertos = 0  
    clear_output(wait=True)   
    print(c)

    #input_documento = doc_consultas_preproc[150]['conteudo']

    # Calcular o vetor TF-IDF para o documento de entrada
    input_vetor_tfidf = np.zeros(num_termos)
    palavras_input = input_documento['conteudo'].lower().split()
    for j, termo in enumerate(vocabulario):
        tf = palavras_input.count(termo.lower()) / len(palavras_input)
        idf = idf_dict[termo]
        input_vetor_tfidf[j] = tf * idf

    # Calcular a similaridade de cosseno entre o documento de entrada e cada documento existente
    similaridades = np.zeros(len(conteudos))
    for i, vetor_tfidf in enumerate(tfidf_matrix):
        similaridades[i] = calcular_similaridade_cosseno(input_vetor_tfidf, vetor_tfidf)

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)

    map_f.append(calcula_map(resultados, input_documento['tema']))
    for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        #print(tema)
        #print(input_documento['tema'])
        if input_documento['tema'] == tema:
            acertos += 1
            
        if rank == 10:
            p10.append(acertos/10.0)
        if rank == 20:
            p20.append(acertos/20.0)
        if rank == 50:
            p50.append(acertos/50.0)
        if rank == 100:
            p100.append(acertos/100.0)
        
    
    if tema_atual != input_documento['tema'] or c == len(doc_consultas_preproc) - 1:
        result_final[tema_atual]['p10'] = mean(p10)
        result_final[tema_atual]['p20'] = mean(p20)
        result_final[tema_atual]['p50'] = mean(p50)
        result_final[tema_atual]['p100'] = mean(p100)     
        result_final[tema_atual]['map'] = mean(map_f)
        p10 = []
        p20 = []
        p50 = []
        p100 = []
        map_f = []           

    tema_atual = input_documento['tema']        


    # if c > 102:
    #     break      


    # for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
    #     print(f"Rank {rank}: Similaridade {similaridade:.4f} - Tema: {tema}")
    #     print(f"Conteúdo: {conteudo}")
    #     print()

1999


In [34]:
result_final

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.5544554455445545,
  'p20': 0.45643564356435645,
  'p50': 0.32613861386138615,
  'p100': 0.24544554455445544,
  'map': 0.21807444032814935},
 'comp.graphics': {'p10': 0.462,
  'p20': 0.406,
  'p50': 0.3078,
  'p100': 0.2468,
  'map': 0.21226083129079779},
 'comp.os.ms-windows.misc': {'p10': 0.414,
  'p20': 0.3625,
  'p50': 0.3106,
  'p100': 0.2679,
  'map': 0.22621798437572277},
 'comp.sys.ibm.pc.hardware': {'p10': 0.407,
  'p20': 0.348,
  'p50': 0.2934,
  'p100': 0.2353,
  'map': 0.1990277891161969},
 'comp.sys.mac.hardware': {'p10': 0.457,
  'p20': 0.3795,
  'p50': 0.3118,
  'p100': 0.261,
  'map': 0.21909450363351446},
 'comp.windows.x': {'p10': 0.465,
  'p20': 0.366,
  'p50': 0.2798,
  'p100': 0.222,
  'map': 0.18900336733467307},
 'misc.forsale': {'p10': 0.331,
  'p20': 0.293,
  'p50': 0.2878,
  'p100': 0.26890000000000003,
  'map': 0.22954135043508944},
 'rec.autos': {'p10': 0.449,
 

In [161]:
result_final

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.7188118811881188,
  'p20': 0.657920792079208,
  'p50': 0.5792079207920792,
  'p100': 0.4873267326732673,
  'map': 0.22119225647265803},
 'comp.graphics': {'p10': 0.505,
  'p20': 0.441,
  'p50': 0.412,
  'p100': 0.3898,
  'map': 0.19187835679356377},
 'comp.os.ms-windows.misc': {'p10': 0.675,
  'p20': 0.6115,
  'p50': 0.5736,
  'p100': 0.5822,
  'map': 0.4248298080553348},
 'comp.sys.ibm.pc.hardware': {'p10': 0.55,
  'p20': 0.452,
  'p50': 0.3778,
  'p100': 0.3629,
  'map': 0.24468762317678025},
 'comp.sys.mac.hardware': {'p10': 0.601,
  'p20': 0.505,
  'p50': 0.436,
  'p100': 0.4139,
  'map': 0.2860355930567415},
 'comp.windows.x': {'p10': 0.543,
  'p20': 0.4675,
  'p50': 0.3898,
  'p100': 0.3479,
  'map': 0.15858726830745754},
 'misc.forsale': {'p10': 0.477,
  'p20': 0.409,
  'p50': 0.3898,
  'p100': 0.3967,
  'map': 0.3125999886875747},
 'rec.autos': {'p10': 0.633,
  'p20': 0.5425,
  'p

In [35]:
import json
with open('results/data_lemma_tfidf_lib_new.json', 'w') as f:
    json.dump(result_final, f)

In [36]:
import pandas as pd

df = pd.DataFrame(result_final)

In [37]:
df

,mini_newsgroups,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,...,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
p10,0,0.554455,0.462000,0.414000,0.407000,0.457000,0.465000,0.331000,0.4490,0.564000,...,0.684000,0.694000,0.398000,0.611000,0.702000,0.878000,0.519000,0.771000,0.459000,0.448485
p20,0,0.456436,0.406000,0.362500,0.348000,0.379500,0.366000,0.293000,0.3310,0.462500,...,0.633000,0.636500,0.301500,0.498000,0.598000,0.857500,0.452500,0.694000,0.382500,0.368182
p50,0,0.326139,0.307800,0.310600,0.293400,0.311800,0.279800,0.287800,0.2350,0.370200,...,0.533400,0.528800,0.225200,0.349000,0.449600,0.827000,0.369600,0.474800,0.277000,0.277374
p100,0,0.245446,0.246800,0.267900,0.235300,0.261000,0.222000,0.268900,0.1889,0.298300,...,0.413600,0.410500,0.183100,0.259500,0.335000,0.738700,0.303000,0.325200,0.213300,0.216364
map,0,0.218074,0.212261,0.226218,0.199028,0.219095,0.189003,0.229541,0.1726,0.265202,...,0.375997,0.383556,0.159615,0.233093,0.307442,0.741019,0.258555,0.310186,0.183428,0.185350


In [38]:
df =df.transpose()

In [39]:
df.drop('mini_newsgroups')

,p10,p20,p50,p100,map
alt.atheism,0.554455,0.456436,0.326139,0.245446,0.218074
comp.graphics,0.462000,0.406000,0.307800,0.246800,0.212261
comp.os.ms-windows.misc,0.414000,0.362500,0.310600,0.267900,0.226218
comp.sys.ibm.pc.hardware,0.407000,0.348000,0.293400,0.235300,0.199028
comp.sys.mac.hardware,0.457000,0.379500,0.311800,0.261000,0.219095
comp.windows.x,0.465000,0.366000,0.279800,0.222000,0.189003
misc.forsale,0.331000,0.293000,0.287800,0.268900,0.229541
rec.autos,0.449000,0.331000,0.235000,0.188900,0.172600
rec.motorcycles,0.564000,0.462500,0.370200,0.298300,0.265202
rec.sport.baseball,0.581000,0.464000,0.361200,0.301000,0.261955


In [41]:
df.describe()

,p10,p20,p50,p100,map
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.516616,0.442410,0.352167,0.282562,0.253868
std,0.182427,0.176621,0.157911,0.135059,0.137500
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.448485,0.362500,0.279800,0.222000,0.189003
50%,0.465000,0.406000,0.311800,0.261000,0.226218
75%,0.611000,0.498000,0.370200,0.303000,0.265202
max,0.878000,0.857500,0.827000,0.738700,0.741019
